In [47]:
import re
import json

import pandas as pd
import numpy as np

from typing import Union
from datetime import datetime, date

In [48]:
df = pd.read_csv("data/raw_csv_files/eu_mrv_shipping_data_2020.csv")

In [49]:
pd.set_option('display.max_columns', None)

In [50]:
df.head()

,IMO Number,Name,Ship type,Reporting Period,Technical efficiency,Port of Registry,Home Port,Ice Class,DoC issue date,DoC expiry date,Verifier Number,Verifier Name,Verifier NAB,Verifier Address,Verifier City,Verifier Accreditation number,Verifier Country,A,B,C,D,Total fuel consumption [m tonnes],Fuel consumptions assigned to On laden [m tonnes],Total CO₂ emissions [m tonnes],CO₂ emissions from all voyages between ports under a MS jurisdiction [m tonnes],CO₂ emissions from all voyages which departed from ports under a MS jurisdiction [m tonnes],CO₂ emissions from all voyages to ports under a MS jurisdiction [m tonnes],CO₂ emissions which occurred within ports under a MS jurisdiction at berth [m tonnes],CO₂ emissions assigned to Passenger transport [m tonnes],CO₂ emissions assigned to Freight transport [m tonnes],CO₂ emissions assigned to On laden [m tonnes],Annual Total Time spent at sea [hours],Annual average Fuel consumption per distance [kg / n mile],Annual average Fuel consumption per transport work (mass) [g / m tonnes · n miles],Annual average Fuel consumption per transport work (volume) [g / m³ · n miles],Annual average Fuel consumption per transport work (dwt) [g / dwt carried · n miles],Annual average Fuel consumption per transport work (pax) [g / pax · n miles],Annual average Fuel consumption per transport work (freight) [g / m tonnes · n miles],Annual average CO₂ emissions per distance [kg CO₂ / n mile],Annual average CO₂ emissions per transport work (mass) [g CO₂ / m tonnes · n miles],Annual average CO₂ emissions per transport work (volume) [g CO₂ / m³ · n miles],Annual average CO₂ emissions per transport work (dwt) [g CO₂ / dwt carried · n miles],Annual average CO₂ emissions per transport work (pax) [g CO₂ / pax · n miles],Annual average CO₂ emissions per transport work (freight) [g CO₂ / m tonnes · n miles],Through ice [n miles],Total time spent at sea [hours],Total time spent at sea through ice [hours],Fuel consumption per distance on laden voyages [kg / n mile],Fuel consumption per transport work (mass) on laden voyages [g / m tonnes · n miles],Fuel consumption per transport work (volume) on laden voyages [g / m³ · n miles],Fuel consumption per transport work (dwt) on laden voyages [g / dwt carried · n miles],Fuel consumption per transport work (pax) on laden voyages [g / pax · n miles],Fuel consumption per transport work (freight) on laden voyages [g / m tonnes · n miles],CO₂ emissions per distance on laden voyages [kg CO₂ / n mile],CO₂ emissions per transport work (mass) on laden voyages [g CO₂ / m tonnes · n miles],CO₂ emissions per transport work (volume) on laden voyages [g CO₂ / m³ · n miles],CO₂ emissions per transport work (dwt) on laden voyages [g CO₂ / dwt carried · n miles],CO₂ emissions per transport work (pax) on laden voyages [g CO₂ / pax · n miles],CO₂ emissions per transport work (freight) on laden voyages [g CO₂ / m tonnes · n miles],Additional information to facilitate the understanding of the reported average operational energy efficiency indicators,Average density of the cargo transported [m tonnes / m³]
0,6602898,OCEAN MAJESTY,Passenger ship,2020,EIV (31.73 gCO₂/t·nm),Madeira,NaN,NaN,01/07/2021,30/06/2022,NaN,Bureau Veritas Certification France,COFRAC,"Le Triangle de l'Arche\n9, cours du Triangle",92937 PARIS LA DEFENSE,4-0076,France,Yes,NaN,NaN,NaN,951.37,NaN,2985.13,1505.53,267.50,367.12,844.98,NaN,NaN,NaN,488.87,132.07,NaN,NaN,NaN,448.73,NaN,414.4,NaN,NaN,NaN,1407.97,NaN,NaN,488.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6703343,EQUALITY,Other ship types,2020,EIV (57.84 gCO₂/t·nm),Beirut,NaN,NaN,30/06/2021,30/06/2022,NaN,Dromon Bureau of Shipping,Hellenic Accreditation System,"2-4 Iakovou Dragatsi street, 18535",Piraeus,1128,Greece,Yes,NaN,NaN,NaN,225.70,NaN,706.78,0.00,401.05,286.49,19.24,NaN,NaN,NaN,603.00,40.75,67.66,NaN,54.28,NaN,NaN,127.62,211.88,NaN,169.97,NaN,NaN,NaN,603.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7037806,IONIAN STAR,Ro-pax

In [51]:
df.applymap(type).apply(pd.value_counts).fillna(0)

,IMO Number,Name,Ship type,Reporting Period,Technical efficiency,Port of Registry,Home Port,Ice Class,DoC issue date,DoC expiry date,Verifier Number,Verifier Name,Verifier NAB,Verifier Address,Verifier City,Verifier Accreditation number,Verifier Country,A,B,C,D,Total fuel consumption [m tonnes],Fuel consumptions assigned to On laden [m tonnes],Total CO₂ emissions [m tonnes],CO₂ emissions from all voyages between ports under a MS jurisdiction [m tonnes],CO₂ emissions from all voyages which departed from ports under a MS jurisdiction [m tonnes],CO₂ emissions from all voyages to ports under a MS jurisdiction [m tonnes],CO₂ emissions which occurred within ports under a MS jurisdiction at berth [m tonnes],CO₂ emissions assigned to Passenger transport [m tonnes],CO₂ emissions assigned to Freight transport [m tonnes],CO₂ emissions assigned to On laden [m tonnes],Annual Total Time spent at sea [hours],Annual average Fuel consumption per distance [kg / n mile],Annual average Fuel consumption per transport work (mass) [g / m tonnes · n miles],Annual average Fuel consumption per transport work (volume) [g / m³ · n miles],Annual average Fuel consumption per transport work (dwt) [g / dwt carried · n miles],Annual average Fuel consumption per transport work (pax) [g / pax · n miles],Annual average Fuel consumption per transport work (freight) [g / m tonnes · n miles],Annual average CO₂ emissions per distance [kg CO₂ / n mile],Annual average CO₂ emissions per transport work (mass) [g CO₂ / m tonnes · n miles],Annual average CO₂ emissions per transport work (volume) [g CO₂ / m³ · n miles],Annual average CO₂ emissions per transport work (dwt) [g CO₂ / dwt carried · n miles],Annual average CO₂ emissions per transport work (pax) [g CO₂ / pax · n miles],Annual average CO₂ emissions per transport work (freight) [g CO₂ / m tonnes · n miles],Through ice [n miles],Total time spent at sea [hours],Total time spent at sea through ice [hours],Fuel consumption per distance on laden voyages [kg / n mile],Fuel consumption per transport work (mass) on laden voyages [g / m tonnes · n miles],Fuel consumption per transport work (volume) on laden voyages [g / m³ · n miles],Fuel consumption per transport work (dwt) on laden voyages [g / dwt carried · n miles],Fuel consumption per transport work (pax) on laden voyages [g / pax · n miles],Fuel consumption per transport work (freight) on laden voyages [g / m tonnes · n miles],CO₂ emissions per distance on laden voyages [kg CO₂ / n mile],CO₂ emissions per transport work (mass) on laden voyages [g CO₂ / m tonnes · n miles],CO₂ emissions per transport work (volume) on laden voyages [g CO₂ / m³ · n miles],CO₂ emissions per transport work (dwt) on laden voyages [g CO₂ / dwt carried · n miles],CO₂ emissions per transport work (pax) on laden voyages [g CO₂ / pax · n miles],CO₂ emissions per transport work (freight) on laden voyages [g CO₂ / m tonnes · n miles],Additional information to facilitate the understanding of the reported average operational energy efficiency indicators,Average density of the cargo transported [m tonnes / m³]
<class 'int'>,11838.0,0.0,0.0,11838.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
<class 'str'>,0.0,11838.0,11838.0,0.0,11838.0,11131.0,2382.0,2084.0,11838.0,11838.0,4993.0,11838.0,11838.0,11838.0,11838.0,11838.0,11838.0,5652.0,3718.0,3854.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11838.0,10575.0,335.0,1412.0,492.0,381.0,11838.0,10575.0,335.0,1412.0,492.0,381.0,0.0,0.0,0.0,1201.0,1156.0,0.0,100.0,0.0,0.0,1201.0,1156.0,0.0,100.0,0.0,0.0,20.0,0.0
<class 'float'>,0.0,0.0,0.0,0.0,0.0,707.0,9456.0,9754.0,0.0,0.0,6845.0,0.0,0.0,0.0,0.0,0.0,0.0,6186.0,8120.0,7984.0,11837.0,11838.0,11838.0,11838.0,11838.0,11838.0,11838.0,11838.0,11838.0,11838.0,11838.0,11838.0,0.0,1263.0,11503.0,10426.0,11346.0,11457.0,0.0,1263.0

In [52]:
for col in df.columns:
    print(df[col].value_counts())

6602898    1
9599078    1
9598842    1
9598854    1
9598866    1
          ..
9356622    1
9356646    1
9356658    1
9356660    1
9910088    1
Name: IMO Number, Length: 11838, dtype: int64
AURORA               4
SKYROS               3
PEGASUS              3
UNDINE               3
PLUTO                3
                    ..
LOTTA AUERBACH       1
LEA AUERBACH         1
DIANA                1
SEA-CARGO EXPRESS    1
PETROTRANS-5904      1
Name: Name, Length: 11682, dtype: int64
Bulk carrier                  3378
Oil tanker                    1877
Container ship                1850
Chemical tanker               1345
General cargo ship            1231
Vehicle carrier                453
Ro-pax ship                    381
Gas carrier                    343
LNG carrier                    265
Ro-ro ship                     239
Refrigerated cargo carrier     144
Other ship types               136
Passenger ship                 111
Container/ro-ro cargo ship      70
Combination carrier         

In [53]:
string_columns = [
    "IMO Number",
    "Name",
    "Ship type",
    "Reporting Period",
    "Technical efficiency",
    "Port of Registry",
    "Home Port",
    "Ice Class",
    "DoC issue date",
    "DoC expiry date",
    "Verifier Number",
    "Verifier Name",
    "Verifier NAB",
    "Verifier Address",
    "Verifier City",
    "Verifier Accreditation number",
    "Verifier Country",
    "A",
    "B",
    "C",
    "D"
]

In [54]:
float_columns = [col for col in df.columns if col not in string_columns]

In [55]:
upper_case_columns = ["Name", "Port of Registry", "Home Port", "Verifier Name", "Verifier NAB","Verifier Address", "Verifier City", "Verifier Country"]

In [56]:
date_columns = ["DoC issue date","DoC expiry date"]

In [57]:
def clean_numerical_data(value: Union[str, int, float]) -> float:
    if value == "Division by zero!":
        return np.nan
    if isinstance(value, str):
        return np.nan
    return round(float(value), 2)

In [58]:
def make_upper_case(value: Union[str, float]) -> str:
    if isinstance(value, str):
        return value.upper()
    return value
    

In [59]:
def extract_technical_efficiency(technical_efficiency: Union[str, float]) -> float:
    
    if technical_efficiency == "Not Applicable":
        return np.nan
    
    if isinstance(technical_efficiency, str):
        try:
            extracted_value = re.findall(r"\d*\.?\d+", technical_efficiency)[0]
            return round(float(extracted_value), 2)
        except IndexError:
            return np.nan
    

In [60]:
def convert_date(date: str) -> str:
    if date == "DoC not issued":
        return np.nan

    date_object = datetime.strptime(date, "%d/%m/%Y").date()
    date_string = datetime.strftime(date_object, "%Y-%m-%d")
    
    return date_string
    

In [61]:
def clean_column(column: str):
    column = re.sub(r"[^a-zA-Z]+", "_", column) 
    column = re.sub(r"_{2,}", "", column)
    column = re.sub(r"_$", "",column)
    return column.lower()

In [62]:
df[float_columns] = df[float_columns].applymap(clean_numerical_data)

In [63]:
df[upper_case_columns] = df[upper_case_columns].applymap(make_upper_case)

In [64]:
df["Technical efficiency"] = df["Technical efficiency"].apply(extract_technical_efficiency)

In [65]:
df[date_columns] = df[date_columns].applymap(convert_date)

In [66]:
df["Reporting Period"] = df["Reporting Period"].apply(lambda x: int(x))

In [67]:
columns_clean = [clean_column(col) for col in df.columns]

In [68]:
df.columns = columns_clean

In [69]:
df.head()

,imo_number,name,ship_type,reporting_period,technical_efficiency,port_of_registry,home_port,ice_class,doc_issue_date,doc_expiry_date,verifier_number,verifier_name,verifier_nab,verifier_address,verifier_city,verifier_accreditation_number,verifier_country,a,b,c,d,total_fuel_consumption_m_tonnes,fuel_consumptions_assigned_to_on_laden_m_tonnes,total_co_emissions_m_tonnes,co_emissions_from_all_voyages_between_ports_under_a_ms_jurisdiction_m_tonnes,co_emissions_from_all_voyages_which_departed_from_ports_under_a_ms_jurisdiction_m_tonnes,co_emissions_from_all_voyages_to_ports_under_a_ms_jurisdiction_m_tonnes,co_emissions_which_occurred_within_ports_under_a_ms_jurisdiction_at_berth_m_tonnes,co_emissions_assigned_to_passenger_transport_m_tonnes,co_emissions_assigned_to_freight_transport_m_tonnes,co_emissions_assigned_to_on_laden_m_tonnes,annual_total_time_spent_at_sea_hours,annual_average_fuel_consumption_per_distance_kg_n_mile,annual_average_fuel_consumption_per_transport_work_mass_g_m_tonnes_n_miles,annual_average_fuel_consumption_per_transport_work_volume_g_m_n_miles,annual_average_fuel_consumption_per_transport_work_dwt_g_dwt_carried_n_miles,annual_average_fuel_consumption_per_transport_work_pax_g_pax_n_miles,annual_average_fuel_consumption_per_transport_work_freight_g_m_tonnes_n_miles,annual_average_co_emissions_per_distance_kg_co_n_mile,annual_average_co_emissions_per_transport_work_mass_g_co_m_tonnes_n_miles,annual_average_co_emissions_per_transport_work_volume_g_co_m_n_miles,annual_average_co_emissions_per_transport_work_dwt_g_co_dwt_carried_n_miles,annual_average_co_emissions_per_transport_work_pax_g_co_pax_n_miles,annual_average_co_emissions_per_transport_work_freight_g_co_m_tonnes_n_miles,through_ice_n_miles,total_time_spent_at_sea_hours,total_time_spent_at_sea_through_ice_hours,fuel_consumption_per_distance_on_laden_voyages_kg_n_mile,fuel_consumption_per_transport_work_mass_on_laden_voyages_g_m_tonnes_n_miles,fuel_consumption_per_transport_work_volume_on_laden_voyages_g_m_n_miles,fuel_consumption_per_transport_work_dwt_on_laden_voyages_g_dwt_carried_n_miles,fuel_consumption_per_transport_work_pax_on_laden_voyages_g_pax_n_miles,fuel_consumption_per_transport_work_freight_on_laden_voyages_g_m_tonnes_n_miles,co_emissions_per_distance_on_laden_voyages_kg_co_n_mile,co_emissions_per_transport_work_mass_on_laden_voyages_g_co_m_tonnes_n_miles,co_emissions_per_transport_work_volume_on_laden_voyages_g_co_m_n_miles,co_emissions_per_transport_work_dwt_on_laden_voyages_g_co_dwt_carried_n_miles,co_emissions_per_transport_work_pax_on_laden_voyages_g_co_pax_n_miles,co_emissions_per_transport_work_freight_on_laden_voyages_g_co_m_tonnes_n_miles,additional_information_to_facilitate_the_understanding_of_the_reported_average_operational_energy_efficiency_indicators,average_density_of_the_cargo_transported_m_tonnes_m
0,6602898,OCEAN MAJESTY,Passenger ship,2020,31.73,MADEIRA,NaN,NaN,2021-07-01,2022-06-30,NaN,BUREAU VERITAS CERTIFICATION FRANCE,COFRAC,"LE TRIANGLE DE L'ARCHE\n9, COURS DU TRIANGLE",92937 PARIS LA DEFENSE,4-0076,FRANCE,Yes,NaN,NaN,NaN,951.37,NaN,2985.13,1505.53,267.50,367.12,844.98,NaN,NaN,NaN,488.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,488.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6703343,EQUALITY,Other ship types,2020,57.84,BEIRUT,NaN,NaN,2021-06-30,2022-06-30,NaN,DROMON BUREAU OF SHIPPING,HELLENIC ACCREDITATION SYSTEM,"2-4 IAKOVOU DRAGATSI STREET, 18535",PIRAEUS,1128,GREECE,Yes,NaN,NaN,NaN,225.70,NaN,706.78,0.00,401.05,286.49,19.24,NaN,NaN,NaN,603.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,603.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7037806,IONIAN STAR,Ro-pax ship,2020,19.40,NaN,NaN,NaN,2021-03-31,2022-06-30,NaN,VERIFAVIA SARL,COFRAC,33 AVENUE DU MAINE,PARIS,4-0596,FRANCE,Yes,NaN,NaN,NaN,1543.32,NaN,4947.88,3713.13,0.00,0.00,1234.60,2968.88,1979.0,NaN,995.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,995.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

## Convert DataFrame to JSON

In [70]:
input_dataframe = df.loc[0]

In [71]:
data_dict = json.loads(input_dataframe.to_json())

In [72]:
data_dict

{'imo_number': 6602898,
 'name': 'OCEAN MAJESTY',
 'ship_type': 'Passenger ship',
 'reporting_period': 2020,
 'technical_efficiency': 31.73,
 'port_of_registry': 'MADEIRA',
 'home_port': None,
 'ice_class': None,
 'doc_issue_date': '2021-07-01',
 'doc_expiry_date': '2022-06-30',
 'verifier_number': None,
 'verifier_name': 'BUREAU VERITAS CERTIFICATION FRANCE',
 'verifier_nab': 'COFRAC',
 'verifier_address': "LE TRIANGLE DE L'ARCHE\n9, COURS DU TRIANGLE",
 'verifier_city': '92937 PARIS LA DEFENSE',
 'verifier_accreditation_number': '4-0076',
 'verifier_country': 'FRANCE',
 'a': 'Yes',
 'b': None,
 'c': None,
 'd': None,
 'total_fuel_consumption_m_tonnes': 951.37,
 'fuel_consumptions_assigned_to_on_laden_m_tonnes': None,
 'total_co_emissions_m_tonnes': 2985.13,
 'co_emissions_from_all_voyages_between_ports_under_a_ms_jurisdiction_m_tonnes': 1505.53,
 'co_emissions_from_all_voyages_which_departed_from_ports_under_a_ms_jurisdiction_m_tonnes': 267.5,
 'co_emissions_from_all_voyages_to_ports